In [6]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV

# Define functions for creating and fitting the LSTM model
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(50, input_shape=input_shape))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def fit_lstm_model(X_train, y_train, batch_size, epochs, input_shape):
    model = create_lstm_model(input_shape)
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)
    return model

def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Load your dataset
# Assuming your dataset is stored in a DataFrame called 'data'
df = pd.read_csv("modeldata.csv", index_col='Date', parse_dates=True)
df.index.freq = "MS"
df['msci_lag1'] = df['msci'].shift(1)
df['sentiment_lag1'] = df['sentiment'].shift(1)
df = df.dropna()

# Initialize an empty DataFrame to store the results
results = pd.DataFrame(columns=['Date', 'Model', 'MAPE', 'RMSE', 'Correct Sign'])

window_size = 36

# Loop over each time window, starting at index 36 up to the second last value of the DataFrame
for i in range(window_size, len(df) - 1):
    train = df.iloc[i - window_size:i]
    test = df.iloc[i:i+1]
    
    # Extract the input features and target variable from the train and test sets
    X_train = train[['returns', 'msci_lag1', 'sentiment_lag1']].values
    y_train = train['returns'].values
    X_test = test[['returns', 'msci_lag1', 'sentiment_lag1']].values
    y_test = test['returns'].values

    # Normalize the data
    scaler = StandardScaler()
    X_train_scaled = np.zeros_like(X_train)
    X_test_scaled = np.zeros_like(X_test)

    for i in range(X_train.shape[1]):
        X_train_scaled[:, i] = scaler.fit_transform(X_train[:, i].reshape(-1, 1)).flatten()
        X_test_scaled[:, i] = scaler.transform(X_test[:, i].reshape(-1, 1)).flatten()

    y_train_scaled = scaler.fit_transform(y_train.reshape(-1, 1))
    y_test_scaled = scaler.transform(y_test.reshape(-1, 1))

    # Reshape the data for LSTM input
    X_train_reshaped = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
    X_test_reshaped = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)
    input_shape = (X_train_reshaped.shape[1], 1)

    # Define the models
    models = {
        'Base': (X_train_reshaped[:, [0]], y_train_scaled),
        'MSCI': (X_train_reshaped[:, [0, 1]], y_train_scaled),
        'Sentiment': (X_train_reshaped[:, [0, 2]], y_train_scaled),
        'Both': (X_train_reshaped, y_train_scaled)
    }

    # Train each model
    for model_name, (X_train_model, y_train_model) in models.items():
        model = fit_lstm_model(X_train_model, y_train_model, batch_size=1, epochs=100, input_shape=input_shape)
        y_pred_scaled = model.predict(X_test_reshaped)
        y_pred = scaler.inverse_transform(y_pred_scaled)
        rmse_score = np.sqrt(mean_squared_error(y_test, y_pred))
        mape_score = mape(y_test, y_pred)
        correct_sign = (np.sign(y_test) == np.sign(y_pred.flatten())).astype(int)

        new_row = pd.DataFrame({
            'Date': [test.index[0]],
            'Model': [model_name],
            'MAPE': [mape_score],
            'RMSE': [rmse_score],
            'Correct Sign': [correct_sign[0]]
        })
        results = pd.concat([results, new_row], ignore_index=True)

# Calculate the average performance measures for each model
average_metrics = results.groupby('Model').agg({
    'MAPE': 'mean',
    'RMSE': 'mean',
    'Correct Sign': 'mean'
}).reset_index()

average_metrics.rename(columns={
    'MAPE': 'Average MAPE',
    'RMSE': 'Average RMSE',
    'Correct Sign': 'Average Accuracy'
}, inplace=True)

average_metrics

/Users/LUCIFER/anaconda3/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step


/Users/LUCIFER/anaconda3/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step


/Users/LUCIFER/anaconda3/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

/Users/LUCIFER/anaconda3/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step


/Users/LUCIFER/anaconda3/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step


,Model,Average MAPE,Average RMSE,Average Accuracy
0,Base,407.265497,0.080079,0.542553
1,Both,9.847945,0.003751,1.000000
2,MSCI,271.988638,0.056587,0.840426
3,Sentiment,281.281782,0.061199,0.819149
